# Lab | Web Scraping

Welcome to the "Books to Scrape" Web Scraping Adventure Lab!

**Objective**

In this lab, we will embark on a mission to unearth valuable insights from the data available on Books to Scrape, an online platform showcasing a wide variety of books. As data analyst, you have been tasked with scraping a specific subset of book data from Books to Scrape to assist publishing companies in understanding the landscape of highly-rated books across different genres. Your insights will help shape future book marketing strategies and publishing decisions.

**Background**

In a world where data has become the new currency, businesses are leveraging big data to make informed decisions that drive success and profitability. The publishing industry, much like others, utilizes data analytics to understand market trends, reader preferences, and the performance of books based on factors such as genre, author, and ratings. Books to Scrape serves as a rich source of such data, offering detailed information about a diverse range of books, making it an ideal platform for extracting insights to aid in informed decision-making within the literary world.

**Task**

Your task is to create a Python script using BeautifulSoup and pandas to scrape Books to Scrape book data, focusing on book ratings and genres. The script should be able to filter books with ratings above a certain threshold and in specific genres. Additionally, the script should structure the scraped data in a tabular format using pandas for further analysis.

**Expected Outcome**

A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`. The function should scrape book data from the "Books to Scrape" website and return a `pandas` DataFrame with the following columns:

**Expected Outcome**

- A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`.
- The function should return a DataFrame with the following columns:
  - **UPC**: The Universal Product Code (UPC) of the book.
  - **Title**: The title of the book.
  - **Price (£)**: The price of the book in pounds.
  - **Rating**: The rating of the book (1-5 stars).
  - **Genre**: The genre of the book.
  - **Availability**: Whether the book is in stock or not.
  - **Description**: A brief description or product description of the book (if available).
  
You will execute this script to scrape data for books with a minimum rating of `4.0 and above` and a maximum price of `£20`. 

Remember to experiment with different ratings and prices to ensure your code is versatile and can handle various searches effectively!

**Resources**

- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)
- [Books to Scrape](https://books.toscrape.com/)


**Hint**

Your first mission is to familiarize yourself with the **Books to Scrape** website. Navigate to [Books to Scrape](http://books.toscrape.com/) and explore the available books to understand their layout and structure. 

Next, think about how you can set parameters for your data extraction:

- **Minimum Rating**: Focus on books with a rating of 4.0 and above.
- **Maximum Price**: Filter for books priced up to £20.

After reviewing the site, you can construct a plan for scraping relevant data. Pay attention to the details displayed for each book, including the title, price, rating, and availability. This will help you identify the correct HTML elements to target with your scraping script.

Make sure to build your scraping URL and logic based on the patterns you observe in the HTML structure of the book listings!


---

**Best of luck! Immerse yourself in the world of books, and may the data be with you!**

**Important Note**:

In the fast-changing online world, websites often update and change their structures. When you try this lab, the **Books to Scrape** website might differ from what you expect.

If you encounter issues due to these changes, like new rules or obstacles preventing data extraction, don’t worry! Get creative.

You can choose another website that interests you and is suitable for scraping data. Options like Wikipedia, The New York Times, or even library databases are great alternatives. The main goal remains the same: extract useful data and enhance your web scraping skills while exploring a source of information you enjoy. This is your opportunity to practice and adapt to different web environments!

In [27]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_books_limited(min_rating, max_price, limit=5):
    base_url = "https://books.toscrape.com/catalogue/page-{}.html"
    current_page = 1

    # Map textual ratings (CSS class) to the number of stars
    rating_map = {
        "One": 1,
        "Two": 2,
        "Three": 3,
        "Four": 4,
        "Five": 5
    }

    all_books = []
    books_collected = 0  # Counter for how many books we've collected so far

    while True:
        url = base_url.format(current_page)
        response = requests.get(url)
        
        # Stop if the page does not exist (404) or is unreachable
        if response.status_code != 200:
            break
        
        soup = BeautifulSoup(response.text, "html.parser")
        
        # Stop if there are no books on this page
        articles = soup.find_all("article", class_="product_pod")
        if not articles:
            break
        
        for article in articles:
            # Link to the book's detail page
            detail_link = article.find("h3").find("a")["href"]
            # The relative URL starts with '../', so we remove it and join it with the base path
            detail_url = "https://books.toscrape.com/catalogue/" + detail_link.replace("../", "")

            # Extract the title
            title = article.find("h3").find("a")["title"].strip()

            # Rating (from the CSS class "star-rating")
            rating_class = article.find("p", class_="star-rating")["class"]
            rating_text = rating_class[-1]
            rating = rating_map.get(rating_text, 0)

            # Price (from the main listing)
            price_str = article.find("p", class_="price_color").text.strip()
            # Remove unwanted characters (e.g., Â, £) before converting
            price_str_clean = price_str.replace("Â", "").replace("£", "")
            price = float(price_str_clean)

            # Navigate to the book's detail page to collect UPC, Availability, Genre, Description
            detail_resp = requests.get(detail_url)
            detail_soup = BeautifulSoup(detail_resp.text, "html.parser")

            # UPC from the information box (table)
            table = detail_soup.find("table", class_="table table-striped")
            upc = None
            availability = None
            if table:
                rows = table.find_all("tr")
                for row in rows:
                    header = row.find("th").text.strip()
                    value = row.find("td").text.strip()
                    if header == "UPC":
                        upc = value
                    elif header == "Availability":
                        availability = "In stock" if "In stock" in value else "Out of stock"

            # Genre: the second-to-last item in the breadcrumb
            breadcrumb = detail_soup.find("ul", class_="breadcrumb").find_all("li")
            if len(breadcrumb) >= 4:
                genre = breadcrumb[-2].get_text(strip=True)
            else:
                genre = "N/A"

            # Description: immediately after <div id="product_description">
            description = None
            product_description_header = detail_soup.find("div", id="product_description")
            if product_description_header:
                desc_paragraph = product_description_header.find_next_sibling("p")
                if desc_paragraph:
                    description = desc_paragraph.text.strip()

            book_data = {
                "UPC": upc,
                "Title": title,
                "Price (£)": price,
                "Rating": rating,
                "Genre": genre,
                "Availability": availability,
                "Description": description
            }

            # Check rating and price criteria
            if rating >= min_rating and price <= max_price:
                # Add the book to the final list
                all_books.append(book_data)
                books_collected += 1

                # Print the data of the book we just collected
                print("Book found:")
                for k, v in book_data.items():
                    print(f"  {k}: {v}")
                print("-" * 50)  # Separator
                
                # Stop if we've reached the book limit
                if books_collected >= limit:
                    break

            # Also stop the main loop if we've reached the limit
            if books_collected >= limit:
                break
        
        if books_collected >= limit:
            break

        current_page += 1  # Move to the next page

    # Transform the data (even if we only have 5) into a DataFrame
    df = pd.DataFrame(all_books)
    return df.reset_index(drop=True)



# Example: rating >= 4, price <= 20, and limit to 5 books
df_books = scrape_books_limited(min_rating=4, max_price=20, limit=5)
print("Final DataFrame:")
df_books.head()


Book found:
  UPC: ce6396b0f23f6ecc
  Title: Set Me Free
  Price (£): 17.46
  Rating: 5
  Genre: Young Adult
  Availability: In stock
  Description: Aaron Ledbetterâs future had been planned out for him since before he was born. Each year, the Ledbetter family vacation on Tybee Island gave Aaron a chance to briefly free himself from his familyâs expectations. When he meets Jonas âLuckyâ Luckett, a caricature artist in town with the traveling carnival, he must choose between the life thatâs been mapped out for him, and Aaron Ledbetterâs future had been planned out for him since before he was born. Each year, the Ledbetter family vacation on Tybee Island gave Aaron a chance to briefly free himself from his familyâs expectations. When he meets Jonas âLuckyâ Luckett, a caricature artist in town with the traveling carnival, he must choose between the life thatâs been mapped out for him, and the chance at true love. ...more
------------------------------------------------

,UPC,Title,Price (£),Rating,Genre,Availability,Description
0,ce6396b0f23f6ecc,Set Me Free,17.46,5,Young Adult,In stock,Aaron Ledbetterâs future had been planned ou...
1,6258a1f6a6dcfe50,The Four Agreements: A Practical Guide to Pers...,17.66,5,Spirituality,In stock,"In The Four Agreements, don Miguel Ruiz reveal..."
2,6be3beb0793a53e7,Sophie's World,15.94,5,Philosophy,In stock,A page-turning novel that is also an explorati...
3,657fe5ead67a7767,Untitled Collection: Sabbath Poems 2014,14.27,4,Poetry,In stock,"More than thirty-five years ago, when the weat..."
4,51653ef291ab7ddc,This One Summer,19.49,4,Sequential Art,In stock,"Every summer, Rose goes with her mom and dad t..."
